## Loader

In [15]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("/home/user/다운로드/test2.pdf")

split_docs = loader.load_and_split()

print(f"문서의 수: {len(split_docs)}")

문서의 수: 67


In [16]:
print(split_docs[0].page_content)

print(split_docs[0].metadata)

석사학위논문 
 
Development Study on The 
Applicability of 4D Seismic 
Monitoring using Anisotropy 
Reverse Time Migration 
Technology Based on Pseudo-
acoustic Wave Equation 
 
 
 
2024년 2월 22일 
 
전 북 대 학 교   대 학 원 
 
환 경 에 너 지 융 합 학 과  
 
김 석 한
{'source': '/home/user/다운로드/test2.pdf', 'page': 0}


In [17]:
print(split_docs)

[Document(page_content='석사학위논문 \n \nDevelopment Study on The \nApplicability of 4D Seismic \nMonitoring using Anisotropy \nReverse Time Migration \nTechnology Based on Pseudo-\nacoustic Wave Equation \n \n \n \n2024년 2월 22일 \n \n전 북 대 학 교   대 학 원 \n \n환 경 에 너 지 융 합 학 과  \n \n김 석 한', metadata={'source': '/home/user/다운로드/test2.pdf', 'page': 0}), Document(page_content='Development Study on The \nApplicability of 4D Seismic \nMonitoring using Anisotropy \nReverse Time Migration \nTechnology Based on Pseudo-\nacoustic Wave Equation \n \n유사음향파동방정식 기반 이방성 역시간구조보정 기술의 \n4차원 탄성파 모니터링 적용성 연구 \n \n2024년 2월 22일 \n \n전 북 대 학 교   대 학 원 \n \n환 경 에 너 지 융 합 학 과  \n \n김 석 한', metadata={'source': '/home/user/다운로드/test2.pdf', 'page': 1}), Document(page_content='Development Study on The \nApplicability of 4D Seismic \nMonitoring using Anisotropy \nReverse Time Migration \nTechnology Based on Pseudo-\nacoustic Wave Equation \n \n지도교수   오 주 원 \n \n이 논문을 공학 석사 학위 논문으로 제출함. \n \n2023년 10월 18일 \n \n전 북 대 학 교   

## Spliter

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=50,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""],
    length_function=len,
)

split_doc = loader.load_and_split(text_splitter=text_splitter)
print(f"RecursiveCharacterTextSplitter \t 사용시 문서의 수: {len(split_doc)}")

RecursiveCharacterTextSplitter 	 사용시 문서의 수: 102


In [21]:
print(split_doc[50].page_content)

perspective, the memory and time required to compute the anisotropic 
wave equation for elliptical anisotropy are almost identical to those 
needed for the isotropic wave equation. This suggests that when


In [22]:
from langchain.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=50)

split_docs = loader.load_and_split(text_splitter=text_splitter)
print(f"TokenTextSplitter \t\t 사용시 문서의 수: {len(split_docs)}")

TokenTextSplitter 		 사용시 문서의 수: 68


## Embedding

In [23]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')

embeddings = []

for doc in split_docs:

    doc_text = doc.page_content  

    embedding = model.encode(doc_text)

    embeddings.append(embedding)

embeddings = np.array(embeddings)

np.save("document_embeddings.npy", embeddings)

In [24]:
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, pipeline
from sklearn.metrics.pairwise import cosine_similarity

document_embeddings = np.load("document_embeddings.npy")

documents = split_docs

model = SentenceTransformer('all-MiniLM-L6-v2')


In [25]:
from sentence_transformers import SentenceTransformer
import numpy as np

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

def find_most_similar_document(query, document_embeddings, documents):

    query_embedding = embedding_model.encode(query)
    query_embedding = query_embedding.reshape(1, -1)

    similarities = cosine_similarity(query_embedding, document_embeddings)

    most_similar_idx = similarities.argmax()

    return documents[most_similar_idx]


## LLM

In [26]:
from transformers import AutoTokenizer, pipeline
from auto_gptq import AutoGPTQForCausalLM
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter

model_name_or_path = "/home/user/anaconda3/envs/alpha/Prac/GPTQ/Llama-2-13B-chat-GPTQ"
model_basename = "model"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
                                           model_basename=model_basename,
                                           use_safetensors=True,
                                           trust_remote_code=True,
                                           device="cuda:0")

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)


Exllama kernel is not installed, reset disable_exllama to True. This may because you installed auto_gptq using a pre-build wheel on Windows, in which exllama_kernels are not compiled. To use exllama_kernels to further speedup inference, you can re-install auto_gptq from source.
CUDA kernels for auto_gptq are not installed, this will result in very slow inference speed. This may because:
1. You disabled CUDA extensions compilation by setting BUILD_CUDA_EXT=0 when install auto_gptq from source.
2. You are using pytorch without CUDA support.
3. CUDA and nvcc are not installed in your device.
skip module injection for FusedLlamaMLPForQuantizedModel not support integrate without triton yet.
The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'C

## RAG

In [31]:
# 사용자의 질문
query = "What is RTM? and how it work?"
most_similar_document = find_most_similar_document(query, document_embeddings, documents)
print("가장 관련 있는 문서:", most_similar_document)

가장 관련 있는 문서: page_content=' - 3 -   \n Kirchhoff migration (Buske, 1999) and Reverse-Time Migration \n(RTM) based on numerical analysis modeling (Baysal et al., 1983) \nappear to be the most promising. While Kirchhoff migration is \npreferred for its computational efficiency and image resolution in \nlarge-scale 3D imaging processes, RTM based on numerical analysis \nmodeling is more suitable for its repeatability and accuracy in imaging, \ndespite its longer computational time due to solving the wave equation \nin both directions. RTM is advantageous for understanding complex \ngeological structures as it accounts for all reflection paths.  \nWith the advancement of high-performance computing technology, \n3D RTM is gradually replacing Kirchhoff migration (Chang and \nMcMechan, 1989; Yoon et al., 2003; Dussaud et al., 2008), and \nvarious techniques are being actively developed to enhance the \nperformance of RTM (Perrone et al., 2011; Crawley et al., 2012; Oh \net al., 2021). RTM com

In [32]:
def generate_answer_with_rag(question, related_document_content, system_message):

    prompt_template = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\n질문: {question}\n관련 정보: {related_document_content}\n[/INST]"
    
    generated_outputs = pipe(prompt_template, num_return_sequences=1) #, max_length=512
    
    generated_answer = generated_outputs[0]['generated_text']
    
    return generated_answer

question = query
related_document_content = most_similar_document
system_message = 'You are analysit.'

answer = generate_answer_with_rag(question, related_document_content, system_message)
print(answer)


[INST] <<SYS>>
You are analysit.
<</SYS>>

질문: What is RTM? and how it work?
관련 정보: page_content=' - 3 -   \n Kirchhoff migration (Buske, 1999) and Reverse-Time Migration \n(RTM) based on numerical analysis modeling (Baysal et al., 1983) \nappear to be the most promising. While Kirchhoff migration is \npreferred for its computational efficiency and image resolution in \nlarge-scale 3D imaging processes, RTM based on numerical analysis \nmodeling is more suitable for its repeatability and accuracy in imaging, \ndespite its longer computational time due to solving the wave equation \nin both directions. RTM is advantageous for understanding complex \ngeological structures as it accounts for all reflection paths.  \nWith the advancement of high-performance computing technology, \n3D RTM is gradually replacing Kirchhoff migration (Chang and \nMcMechan, 1989; Yoon et al., 2003; Dussaud et al., 2008), and \nvarious techniques are being actively developed to enhance the \nperformance of RTM (